<a href="https://colab.research.google.com/github/Tarikouha/TASKS_NLP_USING_HUGGINGFACE/blob/main/TASKS_NLP_USING_HUGGINGFACE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##NLP Tasks with Transformers using HuggingFace

Sentiment analysis 

Text generation

Name entity recognition (NER)

Question answering

Filling masked text

Summarization

Translation

Feature extraction


**Install packages**

In [1]:
pip install transformers sentencepiece

     |████████████████████████████████| 2.6 MB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 37.0 MB/s 
     |████████████████████████████████| 895 kB 49.0 MB/s 
     |████████████████████████████████| 3.3 MB 48.8 MB/s 
     |████████████████████████████████| 636 kB 27.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## **`DataSet`**

In [2]:
texts = [      
            ["stay safe from covid 19", "English"],
            ["Italy won the Euro 2020 final","English"],
            ["احمي نفسك من كوفيد 19" , "Arabic"],
            ["فازت إيطاليا بنهائي يورو 2020", "Arabic"],
            ["protégez vous contre covid 19", "French"],
            ["L'Italie a remporté la finale de l'Euro 2020", "French"]
]

print(texts)

[['stay safe from covid 19', 'English'], ['Italy won the Euro 2020 final', 'English'], ['احمي نفسك من كوفيد 19', 'Arabic'], ['فازت إيطاليا بنهائي يورو 2020', 'Arabic'], ['protégez vous contre covid 19', 'French'], ["L'Italie a remporté la finale de l'Euro 2020", 'French']]


In [3]:
from transformers import pipeline,AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic")
model = AutoModel.from_pretrained("akhooli/gpt2-small-arabic")

Some weights of the model checkpoint at akhooli/gpt2-small-arabic were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
my_classifier_using_pipeline = pipeline('sentiment-analysis' ,  model='akhooli/xlm-r-large-arabic-sent')

Some weights of the model checkpoint at akhooli/xlm-r-large-arabic-sent were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## **Preprocessing data**

In [5]:
import string
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation  #all punctuations 

def preprocessing_data(text , type):
    #remove punctuations
 
  
    if type == "Arabic":

   
      #remove longation and stopwords
      text = re.sub("[إأآا]", "ا", text)
      text = re.sub("ى", "ي", text)
      text = re.sub("ؤ", "ء", text)
      text = re.sub("ئ", "ء", text)
      text = re.sub("ة", "ه", text)
      text = re.sub("گ", "ك", text)

    
    elif type == "English":

      text = text.lower()
      text = ''.join([item for item in text if item not in punctuations ])
      
 
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## **Text classification (Sentiments Analysis) - English, French and Arabic languages**

In [6]:
for text,language in texts:
  res = my_classifier_using_pipeline(preprocessing_data(text , language))[0]
  res['label'] = "Positive" if res['label'] == 'LABEL_2' else ("Negative" if res['label'] == 'LABEL_1'  else "Mixed")
  print("'{}' is {} {} text with occuracy = {}%".format(text,res['label'],language,round(res['score']*100, 2)))

'stay safe from covid 19' is Mixed English text with occuracy = 39.89%
'Italy won the Euro 2020 final' is Positive English text with occuracy = 46.24%
'احمي نفسك من كوفيد 19' is Negative Arabic text with occuracy = 43.62%
'فازت إيطاليا بنهائي يورو 2020' is Positive Arabic text with occuracy = 39.26%
'protégez vous contre covid 19' is Negative French text with occuracy = 39.64%
'L'Italie a remporté la finale de l'Euro 2020' is Positive French text with occuracy = 46.01%


## **Text generation (English, French and Arabic languages)**

In [7]:
from transformers import pipeline, set_seed
generator_english_text = pipeline('text-generation', model='gpt2')
generator_arabic_text = pipeline('text-generation', model='akhooli/gpt2-small-arabic')
generator_french_text = pipeline('text-generation', model='dbddv01/gpt2-french-small')
set_seed(42)

In [8]:
for text , lang in texts:
  if lang == "Arabic":
    print(generator_arabic_text(text , max_length=30, num_return_sequences=5))
  elif lang == "English":
    print(generator_english_text(text , max_length=30, num_return_sequences=5))
  else:
    print(generator_french_text(text , max_length=30, num_return_sequences=5))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'stay safe from covid 19:41:29.967000 <Gre goree> http://www.reddit.com/r/P'}, {'generated_text': "stay safe from covid 19:44 < andytoshi> what's a way to protect against such a risk? 19:44 < gmaxwell>"}, {'generated_text': 'stay safe from covid 19:39:16 <zhanmtz_: Yeah yeah 20:01:33 <rjri> its probably'}, {'generated_text': 'stay safe from covid 19:46 < rickgrin_> what about lings? 19:46 < warren> he is being very'}, {'generated_text': 'stay safe from covid 19:11:28 <DarkLord12> <marion_champion> we now have a reason? if you'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Italy won the Euro 2020 final\n\nThe second stage of the three-day Euro 2020 series takes place in Marseille on the 4th of November'}, {'generated_text': 'Italy won the Euro 2020 final against Germany during the pre-match press conference.\n\nThe club then took the opportunity of their international call-up'}, {'generated_text': 'Italy won the Euro 2020 final with a victory against Denmark, who beat France, who won the competition in Athens. Two years ago, Spain went 2'}, {'generated_text': 'Italy won the Euro 2020 final.\n\n"You might have noticed this year that our teams are all moving very well to try to get stronger,"'}, {'generated_text': 'Italy won the Euro 2020 final in France last November, winning and securing the No.1 jersey with France having won four of its last six.\n'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'احمي نفسك من كوفيد 19، ومن الواضح أن هناك حاجة إلى أدوية المضادة للفيروسات لعلاج أعراض فيروس كورونا. في 17 مايو، أعلن المركز الوطني للسرطان في'}, {'generated_text': 'احمي نفسك من كوفيد 19 حتى إشعار آخر حسب مصدر عسكري: . وفي مارس، ألغت الحكومة التركية محافظة هاتاي بعد عدة أشهر من إعلان حالة الطوارئ'}, {'generated_text': 'احمي نفسك من كوفيد 19. تم تأكيد إصابة 19 شخصا فقط وهم من النساء في المستشفيات. بينما كانت امرأة في السادسة مساء ذكرت في حسابها الخاص أنها'}, {'generated_text': 'احمي نفسك من كوفيد 19 . ثم بدأت حركة التضامن في الساعة العاشرة مساء يوم 8 أبريل، وامتدت مسيرة أكثر من 8 كيلومترات، بعد أن شهدت إصابة'}, {'generated_text': 'احمي نفسك من كوفيد 19، فإن الفيروس الذي يؤثر على الجهاز التنفسي يمكن أن ينتشر عن طريق الهواء الطلق ومن خلال السعال، ويمكن أيضا أن ينتشر عن'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'فازت إيطاليا بنهائي يورو 2020 ، على الرغم من أن البرازيل ستتلقى هذا الطلب، ولم يكن لديها أية قيود. هناك أكثر من 20 بلد'}, {'generated_text': 'فازت إيطاليا بنهائي يورو 2020 لمدة 25 يوما فقط، من حيث نتائج الإحصاء، في حين حصل الفائز بجائزة سان فرانسيسكو كرونيكل على رقم قياسي'}, {'generated_text': 'فازت إيطاليا بنهائي يورو 2020. وقد سجلت إسبانيا 2.2 مليار يورو من الناتج المحلي الإجمالي، أي ما يعادل 2.8 مليار يورو'}, {'generated_text': 'فازت إيطاليا بنهائي يورو 2020. كان موضوع البرنامج عبارة عن شراكة شراكة بين شركة روكستارز الفرنسية وشركة سوني، وهي شركة في مجموعة روك'}, {'generated_text': 'فازت إيطاليا بنهائي يورو 2020 20 في حين أن لديها أقل من 100 من الشركات بما في ذلك شركة ، التي تقدم المنتجات ذات الملكية في الخارج'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'protégez vous contre covid 19/20 avec moi et vous. »(en) George Eliot, The American Notes, New York,'}, {'generated_text': 'protégez vous contre covid 19 ». Le contrat avec les autres sociétés change en 2002 lorsque l’entreprise se voit renoncer.(en)'}, {'generated_text': 'protégez vous contre covid 19. »Lors de la préparation du Tournoi des Six Nations, les Strokeens participent à la préparation'}, {'generated_text': 'protégez vous contre covid 19,6 GIKC ,. Les membres du gouvernement du Québec sont en désaccord avec le projet'}, {'generated_text': "protégez vous contre covid 1912 de l'argent des Juifs. »Pour l'époque contemporaine, la ville s'est installée"}]
[{'generated_text': "L'Italie a remporté la finale de l'Euro 2020.Sur le plan économique, le secteur dans le domaine de la viande est d'inspiration"}, {'generated_text': "L'Italie a remporté la finale de l'Euro 2020 du jeune-duc de Belgique et un match de préparation pour la suite. Mais la Coupe"}, {'generat

## **Name entity recognition (NER) English, French and Arabic languages**

In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

ner_english_recognition = pipeline("ner", model="dslim/bert-base-NER", tokenizer="dslim/bert-base-NER")
ner_arabic_recognition = pipeline("ner", model="hatmimoha/arabic-ner", tokenizer="hatmimoha/arabic-ner")
ner_french_recognition = pipeline("ner", model="gilf/french-postag-model", tokenizer="gilf/french-postag-model")

In [10]:
for text , lang in texts:
  if lang == "Arabic":
    print(ner_arabic_recognition(text))
  elif lang == "English":
    print(ner_english_recognition(text))
  else:
    print(ner_french_recognition(text))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]
[{'entity': 'B-LOC', 'score': 0.99983245, 'index': 1, 'word': 'Italy', 'start': 0, 'end': 5}, {'entity': 'B-MISC', 'score': 0.9972825, 'index': 4, 'word': 'Euro', 'start': 14, 'end': 18}, {'entity': 'I-MISC', 'score': 0.99892485, 'index': 5, 'word': '2020', 'start': 19, 'end': 23}]
[{'entity': 'LABEL_12', 'score': 0.99964976, 'index': 1, 'word': 'احم', 'start': 0, 'end': 3}, {'entity': 'LABEL_12', 'score': 0.99955195, 'index': 2, 'word': '##ي', 'start': 3, 'end': 4}, {'entity': 'LABEL_12', 'score': 0.9993697, 'index': 3, 'word': 'نفسك', 'start': 5, 'end': 9}, {'entity': 'LABEL_12', 'score': 0.99978465, 'index': 4, 'word': 'من', 'start': 10, 'end': 12}, {'entity': 'LABEL_17', 'score': 0.9978187, 'index': 5, 'word': 'كوف', 'start': 13, 'end': 16}, {'entity': 'LABEL_18', 'score': 0.58369833, 'index': 6, 'word': '##يد', 'start': 16, 'end': 18}, {'entity': 'LABEL_18', 'score': 0.99593383, 'index': 7, 'word': '19', 'start': 19, 'end': 21}]
[{'entity': 'LABEL_12', 'score': 0.9993557, 'inde

## **Question answering**

In [12]:
from transformers import pipeline

##### English context
question = "What is the capital of the Morocco?"
context = r"Morocco, a North African country bordering the Atlantic Ocean and Mediterranean Sea, is distinguished by its Berber, Arabian and European cultural influences. Marrakesh’s medina, a mazelike medieval quarter, offers entertainment in its Djemaa el-Fna square and souks (marketplaces) selling ceramics, jewelry and metal lanterns. The capital Rabat’s Kasbah of the Udayas is a 12th-century royal fort overlooking the water."
# Generating an answer to the question in context
qa_english_sentence = pipeline("question-answering")
answer = qa_english_sentence(question=question, context=context)
# Print the answer
print(f"English Question: {question}")
print(f"English Answer: '{answer['answer']}' with accuracy {answer['score']}")

##### French context
qa_french_sentence = pipeline('question-answering', model='fmikaelian/camembert-base-squad', tokenizer='fmikaelian/camembert-base-squad')
question_fr = "Quelle est la capitale du Maroc ?"
context_fr = "Le Maroc est un pays du nord-ouest de l'Afrique. Sa longue côte donnant sur l'océan Atlantique se termine au cap Spartel, limite occidentale du détroit de Gibraltar et où débute le littoral méditerranéen. Au sud du Maroc se trouve le territoire contesté Sahara occidental, revendiqué et contrôlé en grande partie par le Maroc. À l'est et au sud-est, le Maroc est limitrophe de l'Algérie. À l'ouest-sud-ouest et à quelque distance de la côte atlantique se situent les îles Canaries tandis qu'à 672 km à l'ouest-nord-ouest du littoral marocain, on rencontre Madère. Au nord du détroit de Gibraltar se trouve l'Espagne. La capitale administrative est Rabat. Parmi les grandes villes remarquables on trouve Casablanca, Agadir, Fès, Marrakech, Meknès, Tanger, Oujda, Nador."
french_answer = qa_french_sentence(question =  question_fr , context = context_fr)
# Print the answer
print(f"French Question: {question_fr}")
print(f"French Answer: '{french_answer['answer']}' with accuracy {french_answer['score']}")

English Question: What is the capital of the Morocco?
English Answer: 'Rabat’s Kasbah of the Udayas' with accuracy 0.7165263891220093
French Question: Quelle est la capitale du Maroc ?
French Answer: ' Rabat.' with accuracy 0.9795011878013611


## **Filling masked text in English,French and Arabic languages**

In [13]:
from transformers import pipeline
english_fill_mask = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking')
french_fill_mask  = pipeline("fill-mask", model="camembert-base", tokenizer="camembert-base")
arabic_fill_mask = pipeline('fill-mask', model='CAMeL-Lab/bert-base-camelbert-ca')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at CAMeL-Lab/bert-base-camelbert-ca were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
print(english_fill_mask ("I have watched this [MASK] and it was awesome"))
print(french_fill_mask("La voiture est <mask>."))
print(arabic_fill_mask("العلماء هم [MASK] ."))

[{'sequence': 'i have watched this movie and it was awesome', 'score': 0.50970458984375, 'token': 3185, 'token_str': 'movie'}, {'sequence': 'i have watched this show and it was awesome', 'score': 0.1295320987701416, 'token': 2265, 'token_str': 'show'}, {'sequence': 'i have watched this one and it was awesome', 'score': 0.0703236311674118, 'token': 2028, 'token_str': 'one'}, {'sequence': 'i have watched this once and it was awesome', 'score': 0.05573119968175888, 'token': 2320, 'token_str': 'once'}, {'sequence': 'i have watched this film and it was awesome', 'score': 0.034685466438531876, 'token': 2143, 'token_str': 'film'}]
[{'sequence': 'La voiture est neuve.', 'score': 0.14571169018745422, 'token': 8665, 'token_str': 'neuve'}, {'sequence': 'La voiture est prête.', 'score': 0.05925729125738144, 'token': 3049, 'token_str': 'prête'}, {'sequence': 'La voiture est immatriculée.', 'score': 0.04936346784234047, 'token': 24128, 'token_str': 'immatriculée'}, {'sequence': 'La voiture est arriv

## **Summarization**

In [1]:
from transformers import pipeline
summarizer = pipeline("summarization")

In [2]:
article_summary = """Tokyo 2020 Olympics
The opening ceremony is at 8pm Tokyo time (9pm AEST/noon BST/7am EDT) on Friday 23 July, but the baseball, softball and football competitions begin on Wednesday 21 July.
Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013. Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed in March 2020 as a result of the COVID-19 pandemic, and is held largely behind closed doors with no public spectators permitted under the state of emergency. Despite being rescheduled for 2021, the event retains the Tokyo 2020 name for marketing and branding purposes. This is the first time that the Olympic Games have been postponed and rescheduled, rather than cancelled. The Summer Paralympics will be held between 24 August and 5 September 2021, 16 days after the completion of the Olympics.
The chief of the Tokyo 2020 organizing committee on Tuesday did not rule out cancelling the Olympics if Covid-19 cases spiked, as more athletes tested positive for the virus and sponsors ditched plans to attend Friday’s opening ceremony."""

context_fr = """Les Jeux olympiques d'été de 2020, officiellement appelés Jeux de la XXXIIe olympiade de l'ère moderne, sont les deuxièmes Jeux d'été organisés par Tokyo, Japon, après ceux de 1964. Initialement prévus du 24 juillet au 9 août 2020, les Jeux olympiques doivent se tenir du 23 juillet au 8 août 2021 (et les Jeux paralympiques du 24 août au 5 septembre 2021).
L'élection de la ville hôte a eu lieu le 7 septembre 2013 à Buenos Aires lors de la 125e session du Comité international olympique avec trois villes en lice pour accueillir les Jeux : Istanbul, Tokyo et Madrid. Il s'agit du plus faible nombre de villes candidates retenues depuis les Jeux olympiques d'été de 1988.
En mars 2020, en raison de la pandémie de Covid-19, le Premier ministre du Japon Shinzō Abe demande le report des Jeux d'un an, soit en 2021, aussitôt entériné par le CIO. Le nom des Jeux reste inchangé : « Jeux olympiques et paralympiques de Tokyo 2020 ». Les Jeux de 1916, 1940 et 1944 avaient été annulés durant les deux guerres mondiales, mais c'est la première fois qu'ils sont reportés.
Un an plus tard, le 20 mars 2021, la sécurité sanitaire n'étant toujours pas garantie, une mesure, également inédite dans l'histoire des JO, est prise : la tenue des Jeux sans spectateurs venant de l'étranger."""

print("Summary:",summarizer(article_summary, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])
print("Summary:",summarizer(context_fr, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Summary:  Opening ceremony is at 8pm Tokyo time (9pm AEST/noon BST/7am EDT) on Friday 23 July . Baseball, softball and football competitions begin on Wednesday 21 July . Summer Paralympics will be held between 24 August and 5 September 2021, 16 days after the Olympics .
Summary:  Les Jeux olympiques d'été de 2020, officiellement appelés Jeux de la XXXIIe olympiade de l'ère moderne, sont les deuxièmes Jeux d'eté organisés par Tokyo, Japon, après ceux de 1964 . Initialement prévus du 24 juillet au 9 août 2020, les Jeux doivent se tenir du 23 Juillet au 8 aoouût 2021 . L'élection de la ville hôte a eu lieu le 7 septem


## **Translation**

In [3]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_ar = "المغرب يترقب 650 ألف جرعة جديدة‬.. والوضعية الوبائية تؤجل المناعة الجماعية"

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Arabic to English
tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


['Morocco is expecting 650,000 new doses, and the pandemic situation is delaying mass immunization.']

## **Feature extraction**

In [4]:
from transformers import pipeline, AutoTokenizer

feature_extraction = pipeline('feature-extraction', model="distilroberta-base", tokenizer="distilroberta-base")
features = feature_extraction("Tokyo 2020 Olympics")

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
print(features[0])
print(len(features[0]))

[[0.0018242504447698593, 0.09638852626085281, 0.020705169066786766, -0.16632939875125885, 0.09770327061414719, -0.08305050432682037, -0.03552544489502907, 0.07181962579488754, 0.06657739728689194, -0.02818349376320839, 0.021820424124598503, 0.04387502744793892, 0.04427746683359146, -0.06435277312994003, 0.07419554889202118, 0.02240433730185032, -0.08539789915084839, 0.028194155544042587, 0.047744911164045334, -0.03470897302031517, -0.03702600300312042, 0.03912283107638359, -0.05080804228782654, 0.059840407222509384, -0.03168957680463791, 0.01791130192577839, 0.10716447979211807, 0.054421354085206985, -0.04383886232972145, 0.03532686084508896, 0.0026847950648516417, -0.02665221132338047, 0.06650148332118988, 0.025575092062354088, -0.009031799621880054, 0.05186879262328148, 0.05463893711566925, 0.04558035731315613, -0.07762440294027328, 0.03120029717683792, -0.004279182292521, 0.03830797225236893, -0.025089586153626442, 0.03746544197201729, 0.039303869009017944, -0.009937458671629429, -0